In [5]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

file_path = 'unique_airport_coordinates.csv'

try:
    airline_coordinates_df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    exit()


In [ ]:
#Generate monthly date ranges
def generate_monthly_date_ranges(start_year, end_year):
    date_ranges = []
    for year in range(end_year, start_year - 1, -1):  # Loop descending years
        for month in range(12, 0, -1):  # Dec to Jan
            start_date = datetime(year, month, 1)
            if month == 12:
                end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
            else:
                end_date = datetime(year, month + 1, 1) - timedelta(days=1)
            date_ranges.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
    return date_ranges

# Set your date range
start_year = 2008
end_year = 2008

# Generate the monthly date ranges
date_ranges = generate_monthly_date_ranges(start_year, end_year)
unique_coordinates = airline_coordinates_df[['longitude_deg', 'latitude_deg']]
save_directory = 'output/'  

# Loop through each month and accumulate for all coordinates
for start_str, end_str in date_ranges:
    year = start_str[:4]  # Extract year from start date
    month = start_str[5:7]  # Extract month from start date
    monthly_data = []  # Store data for all coordinates for this month

    for index, row in unique_coordinates.iterrows():
        latitude = row['latitude_deg']
        longitude = row['longitude_deg']

        url = f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date={start_str}&end_date={end_str}&hourly=temperature_2m,wind_speed_10m,precipitation,rain,snowfall,visibility"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            hourly_data = data.get('hourly', {})

            if hourly_data:
                df = pd.DataFrame(hourly_data)
                df['latitude_deg'] = latitude
                df['longitude_deg'] = longitude
                df['start_date'] = start_str
                df['end_date'] = end_str
                monthly_data.append(df)

            else:
                print(f"Error: 'hourly' key not found in the API response for coordinates: {latitude}, {longitude}, date range: {start_str} to {end_str}")

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for coordinates {latitude}, {longitude}: {e}")
            continue  

        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            continue  

    # save to csv for each month
    if monthly_data:
        combined_df = pd.concat(monthly_data, ignore_index=True)
        file_path = f"{save_directory}weather_data_{year}_{month}.csv"
        combined_df.to_csv(file_path, index=False)
        print(f"Data saved for {year}-{month} across all coordinates.")
    else:
        print(f"No data retrieved for {year}-{month}.")



In [3]:
# # Function to generate yearly date ranges
# def generate_yearly_date_ranges(start_date, end_date):
#     ranges = []
#     current_date = start_date

#     while current_date < end_date:
#         # Set the next range to be 1 year ahead
#         next_year = current_date.replace(year=current_date.year + 1, month=1, day=1)
#         if next_year > end_date:
#             next_year = end_date + timedelta(days=1)  # Ensure end date is not exceeded

#         ranges.append((current_date, next_year - timedelta(days=1)))
#         current_date = next_year

#     return ranges

# # Set your date range
# start_date = datetime(2008, 12, 31)
# end_date = datetime(1989, 1, 1)

# # Generate the yearly date ranges
# date_ranges = generate_yearly_date_ranges(start_date, end_date)

# # Generate the yearly date ranges
# date_ranges = generate_yearly_date_ranges(start_date, end_date)
# unique_coordinates = airline_coordinates_df[['longitude_deg', 'latitude_deg']]
# save_directory = 'output/' 

# # Loop through each yearly date range and accumulate data for all coordinates
# for start, end in date_ranges:
#     start_str = start.strftime('%Y-%m-%d')
#     end_str = end.strftime('%Y-%m-%d')
#     year = start.year
#     yearly_data = []  

#     # Loop through each coordinate
#     for index, row in unique_coordinates.iterrows():
#         latitude = row['latitude_deg']
#         longitude = row['longitude_deg']

#         # Make the API call for the current coordinate and date range
#         url = f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date={start_str}&end_date={end_str}&hourly=temperature_2m,wind_speed_10m,precipitation,rain,snowfall,visibility"

#         try:
#             response = requests.get(url)
#             response.raise_for_status()
#             data = response.json()
#             hourly_data = data.get('hourly', {})

#             if hourly_data:
#                 df = pd.DataFrame(hourly_data)
#                 df['latitude_deg'] = latitude
#                 df['longitude_deg'] = longitude
#                 df['start_date'] = start_str
#                 df['end_date'] = end_str
#                 yearly_data.append(df)

#             else:
#                 print(f"Error: 'hourly' key not found in the API response for coordinates: {latitude}, {longitude}, date range: {start_str} to {end_str}")

#         except requests.exceptions.RequestException as e:
#             print(f"Error fetching data for coordinates {latitude}, {longitude}: {e}")
#             continue  # Continue to the next coordinate

#         except Exception as e:
#             print(f"An unexpected error occurred: {e}")
#             continue  # Continue to the next coordinate

#     # save to csv for each year
#     if yearly_data:
#         combined_df = pd.concat(yearly_data, ignore_index=True)
#         file_path = f"{save_directory}weather_data_{year}.csv"
#         combined_df.to_csv(file_path, index=False)
#         print(f"Data saved for {year} across all coordinates.")
#     else:
#         print(f"No data retrieved for {year}.")
